# Fake News Detection

## Task 2

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from colorama import Fore, Style

In [2]:
data = pd.read_csv('train_data_mal_fake_detect.csv')

In [36]:
data['Label'].unique()

array(['FALSE     ', 'HALF TRUE', 'FALSE', 'MOSTLY FALSE', 'PARTLY FALSE',
       'MOSTLY TRUE'], dtype=object)

In [3]:
# Fill missing values with an empty string
data['News'] = data['News'].fillna('')

In [4]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['News'])
y = data['Label']

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42)

### Logistic Regression

In [8]:
param_grid_lr = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag']}

In [9]:
lr_classifier = LogisticRegression(max_iter=1000, random_state=42)
grid_search_lr = GridSearchCV(lr_classifier, param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X_train, y_train)

C:\Users\HP\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100], 'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [10]:
# Predict on the development set
best_classifier_lr = grid_search_lr.best_estimator_
lr_predictions = best_classifier_lr.predict(X_dev)
lr_accuracy = accuracy_score(y_dev, lr_predictions)

### Random Forest

In [11]:
param_grid_rf = {'n_estimators': [50, 100, 200],
              'max_depth': [None, 10, 20, 30],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4]}

In [12]:
rf_classifier = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf_classifier, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

C:\Users\HP\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [13]:
# Predict on the development set
best_classifier_rf = grid_search.best_estimator_
rf_predictions = best_classifier_rf.predict(X_dev)
rf_accuracy = accuracy_score(y_dev, rf_predictions)

### SVM

In [14]:
svm_classifier = SVC(kernel='rbf', random_state=42)
svm_classifier.fit(X_train, y_train)
svm_predictions = svm_classifier.predict(X_dev)
svm_accuracy = accuracy_score(y_dev, svm_predictions)

### Naive Bayes

In [15]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_dev)
nb_accuracy = accuracy_score(y_dev, nb_predictions)

### Result and Inference

In [16]:
print("Logistic Regression Accuracy:", lr_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("Support Vector Machine Accuracy:", svm_accuracy)
print("Naive Bayes Accuracy:", nb_accuracy)

Logistic Regression Accuracy: 0.658682634731
Random Forest Accuracy: 0.658682634731
Support Vector Machine Accuracy: 0.658682634731
Naive Bayes Accuracy: 0.652694610778


### Applying SVM in test data which is without label.

In [17]:
test_without_label = pd.read_csv('test-data.csv')

In [18]:
test_without_label['News'] = test_without_label['News'].fillna('')

In [19]:
x = vectorizer.transform(test_without_label['News'])

In [20]:
predictions_without_label_svm = svm_classifier.predict(x)

In [21]:
results_without_label_svm = pd.DataFrame({
    'ID': test_without_label['ID'],
    'News': test_without_label['News'],
    'Predicted': predictions_without_label_svm
})

In [40]:
results_without_label_svm=pd.read_csv('output_file_task2.csv')

In [41]:
df=results_without_label_svm

In [42]:
df

,ID,News,Predicted
0,FAKE_3000,ചന്ദനക്കുറിയണിഞ്ഞ് വിഎസ് അച്യുതാനന്ദൻ.,FALSE
1,FAKE_3001,നെറ്റിയില്‍ ചന്ദനക്കുറിയിട്ട് വിഎസ് അച്യുതാനന്...,FALSE
2,FAKE_3002,മുഖ്യമന്ത്രി പിണറായി വിജയനെതിരെ വിഎസ് അച്യുതാന...,FALSE
3,FAKE_3003,വിശ്വസിച്ച് കൂടെ നിന്ന സഖാവ് അലനേയും സഖാവ് താഹ...,FALSE
4,FAKE_3004,പിണറായി വിജയനെപ്പറ്റി വിഎസ് അച്യുതാനന്ദൻ തരംതാ...,FALSE
5,FAKE_3005,വി എസ് അച്യുതാനന്ദൻ സുരേഷ് ഗോപിക്ക് പിന്തുണ പ്...,FALSE
6,FAKE_3006,കീർത്തി സുരേഷ് ഫർഹാൻ എന്ന മുസ്ലിം യുവാവിനെ കല്...,FALSE
7,FAKE_3007,യെമനില്‍ പുരുഷന്‍മാര്‍ക്ക് രണ്ട് വിവാഹം നിര്‍ബ...,FALSE
8,FAKE_3008,ബിഹാറിൽ എട്ടു വയസുകാരിയുടെ വിവാഹം നടന്നു,FALSE
9,FAKE_3009,ബഹ്‌റൈനിലെ ഇസ്രയേല്‍ എംബസിക്ക് പലസ്തീന്‍ അനുകൂ...,FALSE


In [43]:
unique_values = df['Predicted'].unique()
print(unique_values)

['FALSE' 'FALSE     ' 'HALF TRUE' 'MOSTLY FALSE']


In [44]:
actual_labels = pd.read_csv('Dataset/FakeNews - Test Dataset2.csv')

In [45]:
unique_values = actual_labels['FactCheck'].unique()
print(unique_values)

['False news' 'PARTLY False news' 'MOSTLY False news' 'HALF TRUE'
 'Mostly False news' 'Half true' 'Partly False news']


In [46]:
merged_df = pd.merge(df, actual_labels, left_on='News', right_on='News')

In [49]:
merged_df

,ID,News,Predicted,S.no,FactCheck,Unnamed: 3
0,FAKE_3000,ചന്ദനക്കുറിയണിഞ്ഞ് വിഎസ് അച്യുതാനന്ദൻ.,FALSE,1,False news,NaN
1,FAKE_3001,നെറ്റിയില്‍ ചന്ദനക്കുറിയിട്ട് വിഎസ് അച്യുതാനന്...,FALSE,2,False news,NaN
2,FAKE_3002,മുഖ്യമന്ത്രി പിണറായി വിജയനെതിരെ വിഎസ് അച്യുതാന...,FALSE,3,False news,NaN
3,FAKE_3003,വിശ്വസിച്ച് കൂടെ നിന്ന സഖാവ് അലനേയും സഖാവ് താഹ...,FALSE,4,False news,NaN
4,FAKE_3004,പിണറായി വിജയനെപ്പറ്റി വിഎസ് അച്യുതാനന്ദൻ തരംതാ...,FALSE,5,False news,NaN
5,FAKE_3005,വി എസ് അച്യുതാനന്ദൻ സുരേഷ് ഗോപിക്ക് പിന്തുണ പ്...,FALSE,6,False news,NaN
6,FAKE_3006,കീർത്തി സുരേഷ് ഫർഹാൻ എന്ന മുസ്ലിം യുവാവിനെ കല്...,FALSE,7,False news,NaN
7,FAKE_3007,യെമനില്‍ പുരുഷന്‍മാര്‍ക്ക് രണ്ട് വിവാഹം നിര്‍ബ...,FALSE,8,False news,NaN
8,FAKE_3008,ബിഹാറിൽ എട്ടു വയസുകാരിയുടെ വിവാഹം നടന്നു,FALSE,9,PARTLY False news,NaN
9,FAKE_3009,ബഹ്‌റൈനിലെ ഇസ്രയേല്‍ എംബസിക്ക് പലസ്തീന്‍ അനുകൂ...,FALSE,10,MOSTLY False news,NaN


In [48]:
original_mapping = {
    'False news': 'FALSE',
    'PARTLY False news': 'PARTLY FALSE',
    'MOSTLY False news': 'MOSTLY FALSE',
    'HALF TRUE': 'HALF TRUE',
    'Mostly False news': 'MOSTLY FALSE',
    'Half true': 'HALF TRUE',
    'Partly False news': 'PARTLY FALSE'
}

predicted_mapping = {
    'FALSE' : 'FALSE',
    'FALSE     ': 'FALSE',
    'HALF TRUE': 'HALF TRUE',
    'MOSTLY FALSE': 'MOSTLY FALSE',
    'PARTLY FALSE': 'PARTLY FALSE',
    'MOSTLY TRUE': 'MOSTLY TRUE'
}

In [50]:
merged_df['original_mapped'] = merged_df['FactCheck'].map(original_mapping)
merged_df['predicted_mapped'] = merged_df['Predicted'].map(predicted_mapping)

In [51]:
correct_predictions = merged_df['original_mapped'] == merged_df['predicted_mapped']
accuracy = correct_predictions.sum() / len(correct_predictions)

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 64.00%
